In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, matplotlib as mpl, seaborn as sns, warnings

%matplotlib inline
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family='NanumGothic')
rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False

plt.style.use('ggplot')
color_pal = ["#F8766D", "#D39200", "#93AA00", "#00BA38", "#00C19F", "#00B9E3", "#619CFF", "#DB72FB"]

warnings.filterwarnings(action='ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
df_raw = pd.read_csv('./data/jeju_data_ver1/201901-202003.csv')

In [3]:
df_raw.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [4]:
df_raw.columns

Index(['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM',
       'HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'CSTMR_CNT', 'AMT', 'CNT'],
      dtype='object')

#### 1, 2, 3월 데이터만 추출

In [5]:
df = df_raw[(df_raw['REG_YYMM'] == 202001) | (df_raw['REG_YYMM'] == 202002) | (df_raw['REG_YYMM'] == 202003)]
df.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
20425415,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,2,1,3,345000,3
20425416,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,3,1903450,3
20425417,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,14,1520500,15
20425418,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,9,1239200,9
20425419,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,606700,4


In [6]:
df.fillna('세종시', inplace=True)

In [7]:
df = df.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']).sum().reset_index()

# 군집 데이터 불러오기

In [8]:
cluster_info = pd.read_csv("./data/class_cluster.csv")

# 1번 군집

In [9]:
class_list = ['관광 민예품 및 선물용품 소매업', 
              '그외 기타 분류안된 오락관련 서비스업',
              '그외 기타 스포츠시설 운영업',
              '기타 수상오락 서비스업',
              '내항 여객 운송업',
              '수산물 소매업',
              '여관업',
              '여행사업',
              '전시 및 행사 대행업',
              '호텔업',
              '휴양콘도 운영업']

In [10]:
'여관업' in class_list

True

In [11]:
df['STD_CLSS_NM'].map(lambda x: x in class_list)

0         False
1         False
2         False
3         False
4         False
          ...  
371400     True
371401     True
371402     True
371403     True
371404     True
Name: STD_CLSS_NM, Length: 371405, dtype: bool

In [12]:
df = df[df['STD_CLSS_NM'].map(lambda x: x in class_list)]

In [13]:
df.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
43,202001,강원,강릉시,관광 민예품 및 선물용품 소매업,20s,1,1,14,179495,16
44,202001,강원,강릉시,관광 민예품 및 선물용품 소매업,20s,2,1,64,623500,66
45,202001,강원,강릉시,관광 민예품 및 선물용품 소매업,40s,1,3,3,98000,3
46,202001,강원,강릉시,관광 민예품 및 선물용품 소매업,40s,2,3,13,453600,16
47,202001,강원,강릉시,관광 민예품 및 선물용품 소매업,50s,1,4,7,367700,7


In [14]:
df.STD_CLSS_NM.unique()

array(['관광 민예품 및 선물용품 소매업', '그외 기타 스포츠시설 운영업', '수산물 소매업', '여관업',
       '전시 및 행사 대행업', '호텔업', '휴양콘도 운영업', '여행사업', '기타 수상오락 서비스업',
       '내항 여객 운송업', '그외 기타 분류안된 오락관련 서비스업'], dtype=object)

In [15]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [16]:
df['year'] = df['REG_YYMM'].apply(lambda x: grap_year(x))
df['month'] = df['REG_YYMM'].apply(lambda x: grap_month(x))
df = df.drop(['REG_YYMM'], axis=1)

# Label Encoding

In [18]:
le = LabelEncoder()

In [19]:
# df.drop(['HOM_SIDO_NM', 'HOM_CCG_NM'], axis=1, inplace=True)

In [20]:
categorical_cols = ['CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'AGE']

In [21]:
# for col in categorical_cols:
#     le = le.fit(df['{}'.format(col)])
#     df['{}'.format(col)] = le.transform( df['{}'.format(col)])

In [18]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df[column] = encoder.transform(df[column])

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34817 entries, 43 to 371404
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   CARD_SIDO_NM  34817 non-null  int32
 1   CARD_CCG_NM   34817 non-null  int32
 2   STD_CLSS_NM   34817 non-null  int32
 3   AGE           34817 non-null  int32
 4   SEX_CTGO_CD   34817 non-null  int64
 5   FLC           34817 non-null  int64
 6   CSTMR_CNT     34817 non-null  int64
 7   AMT           34817 non-null  int64
 8   CNT           34817 non-null  int64
 9   year          34817 non-null  int64
 10  month         34817 non-null  int64
dtypes: int32(4), int64(7)
memory usage: 2.7 MB


In [20]:
df.STD_CLSS_NM.unique()

array([ 0,  2,  5,  6,  8,  9, 10,  7,  3,  4,  1])

# Preprocessing

In [21]:
# 변수명 지정
X_cols = list(df.columns)
X_cols.remove('AMT')

In [22]:
X = df.drop(['CSTMR_CNT','AMT','CNT'], axis=1)
y = np.log1p(df['AMT'])

In [23]:
# X 변수 Scale 적용
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [25]:
X_train

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,year,month
125235,16,197,6,3,1,3,2020,1
323307,8,52,5,4,1,3,2020,3
275686,1,151,6,2,1,1,2020,3
100705,12,111,6,3,1,2,2020,1
13642,1,44,9,3,1,4,2020,1
...,...,...,...,...,...,...,...,...
191790,7,77,6,1,1,1,2020,2
64824,7,47,5,4,1,4,2020,1
308798,6,157,5,2,2,3,2020,3
314990,7,85,8,2,2,2,2020,3


# Light Gradient Boosting Machine

In [26]:
import lightgbm as lgb

In [27]:
train_ds = lgb.Dataset(X_train, label=y_train)
val_ds = lgb.Dataset(X_test, label=y_test)

In [28]:
params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

In [29]:
model = lgb.train(params,
                  train_ds,
                  10000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.32584
[200]	valid_0's rmse: 1.20859
[300]	valid_0's rmse: 1.14416
[400]	valid_0's rmse: 1.10069
[500]	valid_0's rmse: 1.06701
[600]	valid_0's rmse: 1.03549
[700]	valid_0's rmse: 1.01187
[800]	valid_0's rmse: 0.992655
[900]	valid_0's rmse: 0.977974
[1000]	valid_0's rmse: 0.966081
[1100]	valid_0's rmse: 0.95563
[1200]	valid_0's rmse: 0.944718
[1300]	valid_0's rmse: 0.933375
[1400]	valid_0's rmse: 0.925066
[1500]	valid_0's rmse: 0.916313
[1600]	valid_0's rmse: 0.910571
[1700]	valid_0's rmse: 0.904962
[1800]	valid_0's rmse: 0.900376
[1900]	valid_0's rmse: 0.895088
[2000]	valid_0's rmse: 0.891328
[2100]	valid_0's rmse: 0.886445
[2200]	valid_0's rmse: 0.881953
[2300]	valid_0's rmse: 0.878372
[2400]	valid_0's rmse: 0.875086
[2500]	valid_0's rmse: 0.87188
[2600]	valid_0's rmse: 0.869086
[2700]	valid_0's rmse: 0.866477
[2800]	valid_0's rmse: 0.864141
[2900]	valid_0's rmse: 0.862204
[3000]	valid_0's rmse: 0.859

In [30]:
X.columns

Index(['CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD',
       'FLC', 'year', 'month'],
      dtype='object')

In [31]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df['STD_CLSS_NM'].unique()
CARD_CCG_NMs  = df['CARD_CCG_NM'].unique()
AGEs          = df['AGE'].unique()
SEX_CTGO_CDs  = df['SEX_CTGO_CD'].unique()
FLCs          = df['FLC'].unique()
years         = [2020]
months        = [4]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for CARD_CCG_NM in CARD_CCG_NMs:
        for STD_CLSS_NM in STD_CLSS_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, CARD_CCG_NM,STD_CLSS_NM,  AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=X.columns)

In [32]:
X.columns

Index(['CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD',
       'FLC', 'year', 'month'],
      dtype='object')

In [33]:
temp

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,year,month
0,0,3,0,1,1,1,2020,4
1,0,3,0,1,1,1,2020,7
2,0,3,0,1,1,3,2020,4
3,0,3,0,1,1,3,2020,7
4,0,3,0,1,1,4,2020,4
...,...,...,...,...,...,...,...,...
5942855,16,169,1,0,2,4,2020,7
5942856,16,169,1,0,2,5,2020,4
5942857,16,169,1,0,2,5,2020,7
5942858,16,169,1,0,2,2,2020,4


In [ ]:
# 예측
pred = model.predict(temp)

In [ ]:

pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
temp

In [ ]:
encoders

In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])